## Importing all the packages

In [283]:
import pandas as pd
import folium
import requests
import numpy as np
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors


## Reading the data frame from the file produced in the homework part-2

In [32]:
full_data=pd.read_csv('full_data.csv').drop('Unnamed: 0', axis=1)
full_data.head(40)


,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


### testing the 

In [27]:
full_data[full_data.PostCode.isin(['M5N'])]

,PostCode,Borough,Neighborhood,Latitude,Longitude
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936


## Creating a new data frame that has all the boroughs with 'Toronto' in the names

In [46]:
Toronto_data=full_data[full_data['Borough'].str.contains('Toronto')]
print('thre are {0} boroughs have the Toronto name'. format ( Toronto_data.shape[0]))
Toronto_data.head()

 
                          

thre are 39 boroughs have the Toronto name


,PostCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### preparing the items for the Four Square API 

In [42]:
CLIENT_ID='D013FASMP0SJ00U3KY2WDL0X3YPTDT3Y4O2D4F0G2BQFPKAM'
CLIENT_SECRET='SAU4UWB3ZM2GXWVFCTOQFFOZWPYS1LORKE1Y34RQTSY4RMDP'
VERSION= '20191220'
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D013FASMP0SJ00U3KY2WDL0X3YPTDT3Y4O2D4F0G2BQFPKAM
CLIENT_SECRET:SAU4UWB3ZM2GXWVFCTOQFFOZWPYS1LORKE1Y34RQTSY4RMDP


### Making a function that calls the Four Square API and get the venues aroud certain coordinate, with radius of 500 meters

In [384]:
def getNearbyVenues(postcodes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postcode, lat, lng in zip(postcodes, latitudes, longitudes):
        print(postcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        print (requests.get(url).json()['response'].keys())
        #print(requests.get(url).json()["response"]['groups'][0])
        
        # return only relevant information for each nearby venue
        venues_list.append([(     
            postcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])    

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])    
 
    
    nearby_venues.columns = ['PostCode', 
                  'PostCode Latitude', 
                  'PostCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

## we want all the venues around all the postcodes, with 'Toronto' in the borough name

In [385]:
 
Toronto_venues = getNearbyVenues(postcodes=Toronto_data['PostCode'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )


M4E
dict_keys(['headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])
M4K
dict_keys(['headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])
M4L
dict_keys(['headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])
M4M
dict_keys(['headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])
M4N
dict_keys(['headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])
M4P
dict_keys(['headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])
M4R
dict_keys(['headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])
M4S
dict_keys(['headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'total

### I saved the results to a file, also checked the shape of the data frame

In [61]:
Toronto_venues.to_csv('Toronto_venues.csv')
Toronto_venues.shape

(1670, 7)

In [98]:
Toronto_venues.head()

,PostCode,PostCode Latitude,PostCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,M4E,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place


### let's count how many different venues we obtained for each postcode

In [309]:
Toronto_venues.groupby('PostCode').count()

,PostCode Latitude,PostCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostCode,,,,,,
M4E,5,5,5,5,5,5
M4K,43,43,43,43,43,43
M4L,19,19,19,19,19,19
M4M,41,41,41,41,41,41
M4N,4,4,4,4,4,4
M4P,8,8,8,8,8,8
M4R,20,20,20,20,20,20
M4S,32,32,32,32,32,32
M4T,4,4,4,4,4,4


In [75]:
print('There are roughly {} different venues in the Toronto area'.format(Toronto_venues['Venue Category'].unique().shape[0]))

There are roughly 233 different venues in the Toronto area


## Doing the One hot encoding to prepare the data for later KMeans clustering

In [387]:
# One hot encoding

Toronto_onehot=pd.get_dummies(Toronto_venues['Venue Category'])
Toronto_onehot.mean()  # This is just testing the result

Afghan Restaurant        0.000597
Airport                  0.000597
Airport Food Court       0.000597
Airport Gate             0.000597
Airport Lounge           0.001194
                           ...   
Video Game Store         0.001194
Vietnamese Restaurant    0.005970
Wine Bar                 0.006567
Wings Joint              0.000597
Yoga Studio              0.004179
Length: 236, dtype: float64

### Creating a new data frame that only has postcode and one hot encoding results, getting ready for the KMeans analysis

In [100]:
Toronto_onehot['PostCode']=Toronto_venues['PostCode']
change_position=[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[:-1])
Toronto_onehot=Toronto_onehot[change_position]
 
Toronto_onehot.head()


,PostCode,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### The data is grouped postcode, then it is ready for the Kmeans analysis

In [194]:
Toronto_grouped=Toronto_onehot.groupby('PostCode').mean().reset_index()
print(Toronto_grouped.shape)
Toronto_grouped.head()
aa=Toronto_grouped.iloc[1].T.iloc[0:]
aa.head()
#Toronto_grouped.iloc[1].index

(38, 234)


PostCode              M4K
Afghan Restaurant       0
Airport                 0
Airport Food Court      0
Airport Lounge          0
Name: 1, dtype: object

### checking out the grouped data

In [388]:
top_five=5
for post in Toronto_grouped['PostCode']:
    temp=Toronto_grouped[Toronto_grouped['PostCode']==post].T.reset_index()
    temp.columns=['Venues','freq']
    temp=temp.iloc[1:,]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})    
    print (post)
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_five))
    

M4E
              Venues  freq
0  Health Food Store  0.14
1        Coffee Shop  0.14
2       Neighborhood  0.14
3                Pub  0.14
4              Trail  0.14
M4K
                   Venues  freq
0        Greek Restaurant  0.21
1             Coffee Shop  0.10
2          Ice Cream Shop  0.07
3  Furniture / Home Store  0.05
4      Italian Restaurant  0.05
M4L
              Venues  freq
0               Park  0.09
1     Sandwich Place  0.09
2        Pizza Place  0.05
3        Coffee Shop  0.05
4  Fish & Chips Shop  0.05
M4M
                Venues  freq
0                 Café  0.10
1          Coffee Shop  0.07
2   Italian Restaurant  0.05
3               Bakery  0.05
4  American Restaurant  0.05
M4N
              Venues  freq
0               Park  0.33
1           Bus Line  0.33
2        Swim School  0.33
3  Afghan Restaurant  0.00
4      Movie Theater  0.00
M4P
              Venues  freq
0     Breakfast Spot  0.14
1               Park  0.14
2     Clothing Store  0.14
3  Food & Drink 

### From the grouped data, creating a data frame that has postcode, and names of top 10 venue categories.

In [358]:
top_venue_number=10

def return_top_venues(row, top_venue_nubmer):
    
    content=row.iloc[1:]
    content_sorted=content.sort_values(ascending=False)
    return content_sorted.index.values[:top_venue_number]

indicators=['st','nd','rd']
columns=['PostCode']

for ind in range(top_venue_number):
    try: columns.append('{}{} most frequent venue'.format(ind+1,indicators[ind+1]))
    
    except: columns.append('{}th most frequent venue'.format(ind+1))
Toronto_sorted=pd.DataFrame()
#print(Toronto_sorted)
Toronto_sorted=pd.DataFrame(columns=columns)
Toronto_sorted['PostCode']=Toronto_grouped['PostCode']
    
print(len(Toronto_grouped.PostCode))
    
for ind in np.arange(len(Toronto_sorted.PostCode)):
    Toronto_sorted.iloc[ind,1:]=return_top_venues(Toronto_grouped.iloc[ind,:], top_venue_number)
  #  print(Toronto_sorted.iloc[ind,:])
    
Toronto_sorted.head()


38


,PostCode,1nd most frequent venue,2rd most frequent venue,3th most frequent venue,4th most frequent venue,5th most frequent venue,6th most frequent venue,7th most frequent venue,8th most frequent venue,9th most frequent venue,10th most frequent venue
0,M4E,Other Great Outdoors,Pizza Place,Health Food Store,Coffee Shop,Trail,Neighborhood,Pub,Doner Restaurant,Diner,Discount Store
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Italian Restaurant,Bookstore,Restaurant,Diner,Pub,Caribbean Restaurant
2,M4L,Sandwich Place,Park,Ice Cream Shop,Food & Drink Shop,Brewery,Burger Joint,Burrito Place,Board Shop,Fast Food Restaurant,Italian Restaurant
3,M4M,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Convenience Store,Brewery,Sandwich Place,Cheese Shop
4,M4N,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


## Now we are using the KMeans to analyze the grouped one hot encoding data

In [359]:
cluster_number=5
grouped_drop=Toronto_grouped.drop('PostCode', axis=1)
kmeans=KMeans(n_clusters=cluster_number, random_state=0).fit(grouped_drop)

kmeans.labels_[0:99]

array([1, 1, 1, 1, 0, 4, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

### a new data frame is created here, with the postcode, borough, neighborhood and all the top venue categories.
### The KMeans labels got inserted to the data frame 
### Note that it was found that the API data is missing one postcode, so I had to get rid of it.

In [360]:

Toronto_sorted.insert(0,'Labels',kmeans.labels_)
Toronto_clustered=Toronto_data.join(Toronto_sorted.set_index('PostCode'), on='PostCode').reset_index().drop('index', axis=1)

# Later I found there is a data missing in the information obtained from Four Square API.  so I will remove the NaN data

Toronto_clustered= Toronto_clustered[~Toronto_clustered.Labels.isna()]



### The Labels values need to be converted to int, so that it can be used to create the map

In [361]:
Toronto_clustered.Labels=Toronto_clustered.Labels.astype(int)
Toronto_clustered.head(40)

,PostCode,Borough,Neighborhood,Latitude,Longitude,Labels,1nd most frequent venue,2rd most frequent venue,3th most frequent venue,4th most frequent venue,5th most frequent venue,6th most frequent venue,7th most frequent venue,8th most frequent venue,9th most frequent venue,10th most frequent venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Other Great Outdoors,Pizza Place,Health Food Store,Coffee Shop,Trail,Neighborhood,Pub,Doner Restaurant,Diner,Discount Store
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Italian Restaurant,Bookstore,Restaurant,Diner,Pub,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1,Sandwich Place,Park,Ice Cream Shop,Food & Drink Shop,Brewery,Burger Joint,Burrito Place,Board Shop,Fast Food Restaurant,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Convenience Store,Brewery,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,4,Park,Hotel,Sandwich Place,Food & Drink Shop,Clothing Store,Breakfast Spot,Gym,Empanada Restaurant,Electronics Store,Eastern European Restaurant
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Coffee Shop,Sporting Goods Shop,Diner,Mexican Restaurant,Dessert Shop,Park,Clothing Store,Chinese Restaurant,Café,Rental Car Location
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Gym,Coffee Shop,Italian Restaurant,Café,Pharmacy,Seafood Restaurant
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,3,Gym,Playground,Park,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,1,Coffee Shop,Pub,Bagel Shop,Fried Chicken Joint,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Light Rail Station


### Marking the postcode on the map, with different colors by their venue category (derived from the one hot encoding data)
### Since I don't where should the map start, I used the mean of the latitudes and longitudes of all Toronto postcode 

In [392]:
# create map
a=Toronto_clustered.Latitude.mean()
b=Toronto_clustered.Longitude.mean()
map_cluster=folium.Map   (location=[a,b], zoom_start=12)


# set color scheme for the clusters
x = np.arange(cluster_number)
ys = [i + x + (i*x)**2 for i in range(cluster_number)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_clustered['Latitude'], Toronto_clustered['Longitude'], Toronto_clustered['PostCode'], Toronto_clustered['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster)
       
map_cluster

#### checking out what is in each category
#### It is obvious that 

In [377]:
Toronto_clustered[Toronto_clustered.Labels==0].drop(['Latitude' , 'Longitude','Labels' ], axis=1)

,PostCode,Borough,Neighborhood,1nd most frequent venue,2rd most frequent venue,3th most frequent venue,4th most frequent venue,5th most frequent venue,6th most frequent venue,7th most frequent venue,8th most frequent venue,9th most frequent venue,10th most frequent venue
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
23,M5P,Central Toronto,"Forest Hill North,Forest Hill West",Park,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


In [379]:
Toronto_clustered[Toronto_clustered.Labels==1].drop(['Latitude' , 'Longitude','Labels' ], axis=1).head()

,PostCode,Borough,Neighborhood,1nd most frequent venue,2rd most frequent venue,3th most frequent venue,4th most frequent venue,5th most frequent venue,6th most frequent venue,7th most frequent venue,8th most frequent venue,9th most frequent venue,10th most frequent venue
0,M4E,East Toronto,The Beaches,Other Great Outdoors,Pizza Place,Health Food Store,Coffee Shop,Trail,Neighborhood,Pub,Doner Restaurant,Diner,Discount Store
1,M4K,East Toronto,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Italian Restaurant,Bookstore,Restaurant,Diner,Pub,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",Sandwich Place,Park,Ice Cream Shop,Food & Drink Shop,Brewery,Burger Joint,Burrito Place,Board Shop,Fast Food Restaurant,Italian Restaurant
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Convenience Store,Brewery,Sandwich Place,Cheese Shop
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Sporting Goods Shop,Diner,Mexican Restaurant,Dessert Shop,Park,Clothing Store,Chinese Restaurant,Café,Rental Car Location


In [380]:
Toronto_clustered[Toronto_clustered.Labels==2].drop(['Latitude' , 'Longitude','Labels' ], axis=1)

,PostCode,Borough,Neighborhood,1nd most frequent venue,2rd most frequent venue,3th most frequent venue,4th most frequent venue,5th most frequent venue,6th most frequent venue,7th most frequent venue,8th most frequent venue,9th most frequent venue,10th most frequent venue
22,M5N,Central Toronto,Roselawn,Garden,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [381]:
Toronto_clustered[Toronto_clustered.Labels==3].drop(['Latitude' , 'Longitude','Labels' ], axis=1)

,PostCode,Borough,Neighborhood,1nd most frequent venue,2rd most frequent venue,3th most frequent venue,4th most frequent venue,5th most frequent venue,6th most frequent venue,7th most frequent venue,8th most frequent venue,9th most frequent venue,10th most frequent venue
8,M4T,Central Toronto,"Moore Park,Summerhill East",Gym,Playground,Park,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
10,M4W,Downtown Toronto,Rosedale,Park,Playground,Trail,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


In [382]:
Toronto_clustered[Toronto_clustered.Labels==4].drop(['Latitude' , 'Longitude','Labels' ], axis=1)

,PostCode,Borough,Neighborhood,1nd most frequent venue,2rd most frequent venue,3th most frequent venue,4th most frequent venue,5th most frequent venue,6th most frequent venue,7th most frequent venue,8th most frequent venue,9th most frequent venue,10th most frequent venue
5,M4P,Central Toronto,Davisville North,Park,Hotel,Sandwich Place,Food & Drink Shop,Clothing Store,Breakfast Spot,Gym,Empanada Restaurant,Electronics Store,Eastern European Restaurant
